# Word Sense Disambiguation
- Evgeny A. Stepanov
- stepanov.evgeny.a@gmail.com

*Recommended Reading*:
- Dan Jurafsky and James H. Martin. [__Speech and Language Processing__ (SLP)](https://web.stanford.edu/~jurafsky/slp3/) (3rd ed. draft)
    - Chapter 18: Word Senses and WordNet
- Steven Bird, Ewan Klein, and Edward Loper. [__Natural Language Processing with Python__ (NLTK)](https://www.nltk.org/book/)
    - Chapter 2: Accessing Text Corpora and Lexical Resources
        - Section 5: WordNet

__Requirements__

- [NLTK](https://www.nltk.org/)
    - WordNet
    - Corpora
        - SemCor
        - Senseval

In [1]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/eas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Take-Aways
- Word Sense Disambiguation
    - Lexical Relations
- Word senses in WordNet
    - Semantic Similarity (in WordNet)
- Dictionary-based Word Sense Disambiguation with WordNet
    - Lesk Algorithm
    - Graph-based Methods
- Supervised Word Sense Disambiguation
    - Feature Extractions for Classification
        - Bag-of-Words
        - Collocational Features
    - Training and Evaluation

## Word Sense Disambiguation
- Human Language is ambiguous
    - Syntacting ambiguity
        - Resolved by POS-tagging
        - Syntactic Parsing
    - Lexical ambiguity
        - Resolved by Word Sense Disambiguation
        - Semantics work at level of word __senses__, not __words__
        
__Example__:
- NOUN
    - 'they pulled the canoe up on the __bank__'
    - 'he cashed a check at the __bank__'
- VERB
    - 'the plane went into a steep __bank__'
    - '__bank__ on your good education'

### Task Variants
- __Lexical sample subtask__: only a small selection of words has to be disambiguated
    - Supervised machine learning: train a classifier for each word
- __All words subtask__: each and every content word in the test corpus has to be disambiguated.
    - Data sparseness issue, can't train a classifier for each word

### Evaluation
Precision, recall, F1-measure against gold standard data

### Lexical Relations
Relation between word senses.

- __Homonymy__: senses are not related
- __Polysemy__: senses are related
- __Metonymy__: a thing or concept is referred to by the name of something closely associated with that thing or concept. (e.g. *Rome* for Italian Government)

- __Synonymy__: senses are identical
- __Antonymy__: senses are opposite
- __Hyponymy__ (specific) and __Hypernymy__ (generic): class-inclusion relationships
- __Meronymy__ (part) and __Holonymy__ (whole): the part-whole relation

## WordNet

[WordNet](https://wordnet.princeton.edu/) is a lexical database of semantic relations between words that links words into semantic relations including synonyms, hyponyms, and meronyms. 

Nouns, verbs, adjectives and adverbs are grouped into sets of cognitive synonyms (synsets), each expressing a distinct concept. Synsets are interlinked by means of conceptual-semantic and lexical relations. 


__Summary__

WordNet is a:
- Graph (4 graphs for each of nouns, verbs, adjectives, and adverbs)
- Nodes are Synsets (synonyms)
- Labeled Edges are Relations between Synsets

    - PART-OF
    - KIND-OF (IS-A)
    - ENTAILMENT
    - ANTONYMY
    
> Senses in WordNet are generally ordered from most to least frequently used, with the most common sense numbered 1.

[WordNet Site](https://wordnet.princeton.edu/documentation/wndb5wn)

In [2]:
# Let's import WordNet
from nltk.corpus import wordnet

In [3]:
# printing senses of a word (including honomymy & polysemy)
senses = wordnet.synsets('bank')
print(senses)

[Synset('bank.n.01'), Synset('depository_financial_institution.n.01'), Synset('bank.n.03'), Synset('bank.n.04'), Synset('bank.n.05'), Synset('bank.n.06'), Synset('bank.n.07'), Synset('savings_bank.n.02'), Synset('bank.n.09'), Synset('bank.n.10'), Synset('bank.v.01'), Synset('bank.v.02'), Synset('bank.v.03'), Synset('bank.v.04'), Synset('bank.v.05'), Synset('deposit.v.02'), Synset('bank.v.07'), Synset('trust.v.01')]


### Synset 
The entity `bank.n.01` is called a __synset__, or "synonym set", a collection of synonymous words (or "lemmas").

The name is composed as `<lemma>.<pos>.<number>` string where: 
- `<lemma>` is the word's morphological stem 
- `<pos>` is one of the module attributes `ADJ`, `ADJ_SAT`, `ADV`, `NOUN` or `VERB` 
- `<number>` is the sense number, counting from `0`

Part-of-speech tags appear as below:

| POS | in Synset Name |
|:----|:---------------|
| `wn.NOUN`    | `n`
| `wn.VERB`    | `v`
| `wn.ADV`     | `r`
| `wn.ADJ`     | `a`
| `wn.ADJ_SAT` | `s` (satelite adjective, ignore)


In [4]:
# it's possible to provide part of speech to filter senses as well
senses = wordnet.synsets('bank', wordnet.NOUN)
print(senses)
print(senses[0].pos())  # part-of-speech tag of a synset

[Synset('bank.n.01'), Synset('depository_financial_institution.n.01'), Synset('bank.n.03'), Synset('bank.n.04'), Synset('bank.n.05'), Synset('bank.n.06'), Synset('bank.n.07'), Synset('savings_bank.n.02'), Synset('bank.n.09'), Synset('bank.n.10')]
n


Each word of a synset can have several meanings, synset represents the single meaning that is common to all words in it. 
Each synset has a __definition__ and __example__ sentences, that can be accessed using `definition()` and `examples()` methods.

In [5]:
print(senses[0].definition())
print(senses[0].examples())

sloping land (especially the slope beside a body of water)
['they pulled the canoe up on the bank', 'he sat on the bank of the river and watched the currents']


### Lemmatization
`wordnet.synsets()` method expects a word to be a __lemma__, i.e. canonical (dictionary) form of a word. In case it does find a word in WordNet, it internally applies morphological transformation rules to strip off affixes untill it finds the form.

```
MORPHOLOGICAL_SUBSTITUTIONS = {
    NOUN: [("s", ""), ("ses", "s"), ("ves", "f"), ("xes", "x"), ("zes", "z"), 
           ("ches", "ch"), ("shes", "sh"), ("men", "man"), ("ies", "y"), ],
    VERB: [("s", ""), ("ies", "y"), ("es", "e"), ("es", ""), 
           ("ed", "e"), ("ed", ""), 
           ("ing", "e"), ("ing", ""), ],
    ADJ: [("er", ""), ("est", ""), ("er", "e"), ("est", "e")],
    ADV: [],
}
```

Those could be applied calling `wordnet.morphy()`.

In [6]:
wordnet.morphy('banked')

'bank'

In [7]:
# Note that only verb synsets are listed
wordnet.synsets('banked') 

[Synset('bank.v.01'),
 Synset('bank.v.02'),
 Synset('bank.v.03'),
 Synset('bank.v.04'),
 Synset('bank.v.05'),
 Synset('deposit.v.02'),
 Synset('bank.v.07'),
 Synset('trust.v.01')]

`wordnet.morphy()` is the basis of the WordNet-based Lemmatizer in NLTK. The Lemmatizer can be used as follows, optionally providing a part-of-speech (default is NOUN).

In [8]:
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
print(lem.lemmatize('banks'))
print(lem.lemmatize('banked', pos=wordnet.VERB))
print(lem.lemmatize('bnked', pos=wordnet.VERB))  # returns the word itself if it cannot find it

bank
bank
bnked


#### Lemmas in WordNet
In WordNet __Lemma__ is a pairing of words with a synset: `bank.n.01` + `bank`.

From a __synset__ we can get:
- all its lemmas (`lemmas()`)
- all its lemma names (`lemma_names()`)

From a __lemma__ we can get:
- its name (`name()`)
- synset it belongs to (`synset()`)

Similar to synsets, we can get all lemmas for a word as well using `lemmas()`.

In [9]:
lemmas = wordnet.lemmas('bank')
print(lemmas)

[Lemma('bank.n.01.bank'), Lemma('depository_financial_institution.n.01.bank'), Lemma('bank.n.03.bank'), Lemma('bank.n.04.bank'), Lemma('bank.n.05.bank'), Lemma('bank.n.06.bank'), Lemma('bank.n.07.bank'), Lemma('savings_bank.n.02.bank'), Lemma('bank.n.09.bank'), Lemma('bank.n.10.bank'), Lemma('bank.v.01.bank'), Lemma('bank.v.02.bank'), Lemma('bank.v.03.bank'), Lemma('bank.v.04.bank'), Lemma('bank.v.05.bank'), Lemma('deposit.v.02.bank'), Lemma('bank.v.07.bank'), Lemma('trust.v.01.bank')]


In [10]:
# Look up lemma directly
lemma = wordnet.lemma('bank.n.01.bank')
print(lemma.name())
print(lemma.synset())

bank
Synset('bank.n.01')


In [11]:
# Get Lemmas of a synset
print(senses[0].lemmas())
print(senses[0].lemma_names())

[Lemma('bank.n.01.bank')]
['bank']


### Lexical Relations beween Synsets

WordNet synsets correspond to abstract concepts that are linked together in a hierarchy from very general (such as `Entity`, `State`, `Event` a.k.a *unique beginners* or *root synsets*) to very specific. 

Hypernymy/Hyponymy relations are used to navigate the taxonomy using `hypernyms()` and `hyponyms()` methods.

- `hypernym_paths()` gets the lists of the hypernym synsets to the root (several paths are possible)
- `root_hypernyms()` gets the root synset
- `hypernym_distances()` get the path(s) from the synset to the root, counting the distance of each node from the initial node on the way

- `max_depth()` returns the length of the longest hypernym path from the synset to the root.
- `min_depth()` returns the length of the shortest hypernym path from the synset to the root.

In [12]:
print(senses[0].hyponyms())
print(senses[0].hypernyms())

[Synset('riverbank.n.01'), Synset('waterside.n.01')]
[Synset('slope.n.01')]


In [13]:
# getting paths to the root of the taxonomy
print(senses[0].hypernym_paths())
# getting hypernyms with distances
print(senses[0].hypernym_distances())
# getting the root node
print(senses[0].root_hypernyms())
print(senses[0].max_depth())
print(senses[0].min_depth())

[[Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('geological_formation.n.01'), Synset('slope.n.01'), Synset('bank.n.01')]]
{(Synset('object.n.01'), 3), (Synset('physical_entity.n.01'), 4), (Synset('bank.n.01'), 0), (Synset('slope.n.01'), 1), (Synset('entity.n.01'), 5), (Synset('geological_formation.n.01'), 2)}
[Synset('entity.n.01')]
5
5


Read about other relations defined for synsets and lemmas in the [NLTK documentation](http://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.wordnet).

__Whole description of WordNet methods and structure is out of the scope of the lab.__

## Lesk Algorithm

> "What we try is to guess the correct word sense by counting overlaps between dictionary definitions of the various senses." 

(Lesk, Michael. "Automatic sense disambiguation using machine readable dictionaries: how to tell a pine cone from an ice cream cone." Proceedings of the 5th Annual International Conference on Systems Documentation. ACM, 1986.)

### Simplified Lesk Algorithm

Kilgarriff and Rosenzweig (2000) [English SENSEVAL](http://www.lrec-conf.org/proceedings/lrec2000/pdf/8.pdf)

```
For each sense s of that word,
    set weight(s) to zero.

Identify set of unique words W in surrounding sentence.

For each word w in W,
    for each sense s,
        if w occurs in the definition or example sentences of s,
            add weight(w) to weight(s).
Choose sense with greatest weight(s)
```

> `weight(w)` is defined as the inverse document frequency (IDF) of the word `w` over the definitions and example sentences in the dictionary. The IDF of a word `w` is computed as `-log(p(w))`, where `p(w)` is estimated as the fraction of dictionary "documents" -- definition or examples -- which contain the word. 

### Lesk Plus Corpus

> LESK-PLUS-CORPUS is as LESK, but also considers the tagged training data, so can be compared with supervised
systems. For each word in the sentence containing the test item, it tests whether `w` occurs in the dictionary entry or corpus instances for each candidate sense.


### Simple Lesk with Equal Weights

If all words are equally weighted, we compute an overlap.
The algorithm becomes simpler.

```
function SIMPLIFIED LESK(word, sentence) returns best sense of word
    best-sense := most frequent sense for word (i.e. first in WordNet)
    max-overlap := 0
    context := set of words in sentence
    for each sense in senses of word do
        signature := set of words in gloss and examples of sense
        overlap := COMPUTE_OVERLAP(signature, context)
        if overlap > max-overlap then
            max-overlap := overlap
            best-sense := sense
    end
return(best-sense)
```

```
COMPUTE OVERLAP returns the number of words in common between two sets.
```

#### Improvements

- Removing stop words
    - IDF makes them weight less in Simplified Lesk by Kilgarriff and Rosenzweig (2000)

### Using Lesk in NLTK
NLTK provide the implementation of the Lesk Algorithm is [`wsd` module](https://www.nltk.org/_modules/nltk/wsd.html).

In [14]:
from nltk.wsd import lesk

sense = lesk('Jane sat on the sloping bank of a river beside the water'.split(), 'bank')
print(sense)
print(sense.definition())

# possible to specify the POS
print(lesk('Jane sat on the sloping bank of a river beside the water'.split(), 
           'bank', 
           pos=wordnet.NOUN))

# possible to specify the synsets to choose from
print(lesk('Jane sat on the sloping bank of a river beside the water'.split(), 
           'bank', 
           synsets=wordnet.synsets('riverbank')))

Synset('bank.n.01')
sloping land (especially the slope beside a body of water)
Synset('bank.n.01')
Synset('riverbank.n.01')


### Alternative Implementations of Lesk in `pywsd`

[`pywsd` library](https://github.com/alvations/pywsd) provides several variants of the Lesk algorithm.



- Original Lesk (Lesk, 1986) -- also *simplified*
- Adapted/Extended Lesk (Banerjee and Pederson, 2002/2003)
- Simple Lesk (with definition, example(s) and hyper+hyponyms)
- Cosine Lesk (use cosines to calculate overlaps instead of using raw counts)

Unfortunatelly, it has some compatibility issues. However, can be consulted for implementations.

### Exercises
Even though NLTK states that it implements Original Lesk Algorithm, in fact it is a Simplified Lesk Algorithm, that doesn't consider examples, and computes overlaps like the original. 

In the original algorithm context is computed differently. Instead of comparing a target word's signature with the context words, the target signature is compared with the signatures of each of the context words.

Implement the Original Lesk Algorithm (modifying NLTK's, see pseudocode in the next section)
- return the first (most frequent sense, if no overlaps)
- remove stopwords
    - `from nltk.corpus import stopwords`
    - `stopwords.words('english')`
- automatically predict pos-tag if not given
    - ignore the case if word appears twice (take first)
- take the majority decision (the sense predicted most frequently)


## Graph-based Methods on WordNet for WSD

### Maximum Relatedness Disambiguation

Pedersen et al. (2003) [Maximizing Semantic Relatedness to Perform Word Sense Disambiguation](https://www.d.umn.edu/~tpederse/Pubs/max-sem-relate.pdf)


```
w = words

foreach sense s[t][i] of target word w[t]$
    set score[i] = 0
    foreach word w[j] in window of context
        skip to next word if j == t

        foreach sense s[j][k] of w[j]
            temp_score[j] = relatedness(s[t][i], s[j][k])

        winning_score = highest score in array temp_score[]

        if (winning_score > threshold)
            set score[i] = score[i] + winning_score
            
return i, such that score[i] >= score[j] , for all j, 1 <= j <= n, n = number of words in sentence
```

#### How do we define relatedness?

- Similar words are near-synonyms: e.g. *car*, *motorcycle*
- Related words can be related any way: e.g. *car*, *fuel*

- Thesaurus-based similarity
    - words have similar definitions (Lesk)
    - words are close to each other in hypernym hiearchy (graph-based)
- Distributional similarity
    - do words apprear in similar distributional contexts
    - __distributional (vector) semantics__

#### Path-based Similarity

Two concepts (senses/synsets) are similar if they are near each other in the thesaurus hierarchy
- have a __short path__ between them (1 + number of edges between nodes)
- path to themselves is has distance of `1`

##### NLTK Path Based Metrics

- `synset1.path_similarity(synset2)`: Return a score denoting how similar two word senses are, based on the __shortest path__ that connects the senses in the is-a (hypernym/hypnoym) taxonomy. The score is in the range 0 to 1, computed as `1/path_length`
- `synset1.lch_similarity(synset2)`: __Leacock-Chodorow Similarity__: Return a score denoting how similar two word senses are, based on the shortest path that connects the senses and the maximum depth of the taxonomy in which the senses occur. The relationship is given as `-log(p/2d)` where `p` is the shortest path length and `d` the taxonomy depth.
- `synset1.wup_similarity(synset2)`: __Wu-Palmer Similarity__: Return a score denoting how similar two word senses are, based on the depth of the two senses in the taxonomy and that of their __Least Common Subsumer__ (most specific ancestor node).

In [15]:
bank_r = wordnet.synsets('bank')[0]
bank_f = wordnet.synsets('bank')[1]
river = wordnet.synsets('river')[0]

print(river.definition())
print(bank_r.definition())
print(bank_f.definition())

print(bank_r.path_similarity(river))
print(bank_f.path_similarity(river))

a large natural stream of water (larger than a creek)
sloping land (especially the slope beside a body of water)
a financial institution that accepts deposits and channels the money into lending activities
0.1111111111111111
0.07692307692307693


In [16]:
print(bank_r.lch_similarity(river))
print(bank_f.lch_similarity(river))

print(bank_r.wup_similarity(river))
print(bank_f.wup_similarity(river))

1.4403615823901665
1.072636802264849
0.3333333333333333
0.14285714285714285


#### Information Content Similarity

- Path-based similarity issues
    - each edge is has equal distance; however nodes high in hierarchy are more abstract
- Better metric
    - each edge has independent cost
    - nodes connected through higher-level (abstract) nodes are less similar

##### Information Content
- Trained on a corpus
- `P(c)` the probability of a concept `c` in a corpus
    $$ P(c) = \frac{\sum_{w \in \text{words}(c)}\text{count}(c)}{N}$$
    where $\text{words}(c)$ is set of all words that are children of concept $c$
- All words are members of the root node (e.g. `Entity`); thus, `P(root) = 1`
- The lower a node in hierarchy, the lower its probability

- Information ($I$) measured in bits as $$I(w) = -log2(P(w))$$
- Information Content $$IC(c) = -log(P(c))$$
- Most Informative Subsumer (Lowest Common Subsumer) $LCS(c_1, c_2)$ is the lowest node in the hierarchy subsuming both $c_1$ and $c_2$

##### NLTK Information Content Based Metrics
- `res_similarity(other, ic)`: __Resnik Similarity__: Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node). Computed as `IC(lcs) = -log(P(lcs))`. Lower is more similar.
- `lin_similarity(other, ic)`: __Lin Similarity__: Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node) and that of the two input Synsets. The relationship is given by the equation `2 * IC(lcs) / (IC(s1) + IC(s2))`.
- `jcn_similarity(other, ic)`: __Jiang-Conrath Similarity__: Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node) and that of the two input Synsets. The relationship is given by the equation `1 / (IC(s1) + IC(s2) - 2 * IC(lcs))`.

In [17]:
# getting pre-computed ic of the semcor corpus (large sense tagged corpus)
from nltk.corpus import wordnet_ic
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

In [18]:
print(bank_r.res_similarity(river, semcor_ic))
print(bank_f.res_similarity(river, semcor_ic))

0.6143639493869085
-0.0


In [19]:
print(bank_r.lin_similarity(river, semcor_ic))
print(bank_f.lin_similarity(river, semcor_ic))

0.07824755637576701
-0.0


In [20]:
print(bank_r.jcn_similarity(river, semcor_ic))
print(bank_f.jcn_similarity(river, semcor_ic))

0.06908770509261253
0.06248754962684728


### Exercise
Extend Lesk algorithm (function) to use similarity metrics instead of just overlaps
- make it a keyword argument to allow different metrics

## Evaluation on Senseval 2

### Senseval Corpus
The Senseval 2 Corpus contains data intended to train word-sense disambiguation classifiers. 
It contains data for four words: `hard`, `interest`, `line`, and `serve`. Let's use `interest` portion to illustrate evaluation.

In [21]:
nltk.download('senseval')

[nltk_data] Downloading package senseval to /Users/eas/nltk_data...
[nltk_data]   Package senseval is already up-to-date!


True

Corpus instances are stored as:
- `context` - POS-tagged context sentence
- `position` - index of the target word in a context sentence
- `senses` - labels

In [22]:
from nltk.corpus import senseval

inst = senseval.instances('interest.pos')[0]

print(inst.position, inst.context, inst.senses)

18 [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')] ('interest_6',)


#### Mapping Senseval Senses to WordNet

Senseval labels are not compatible with WordNet 3.0; thus, let's manually create a mapping.

__Senses for *interest* in Longman Dictionary__
- Sense 1 =  361 occurrences (15%) - readiness to give attention
- Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
- Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
- Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
- Sense 5 =  500 occurrences (21%) - a share in a company or business
- Sense 6 = 1252 occurrences (53%) - money paid for the use of money

In [23]:
# definitions of "interest"'s synsets in WordNet
iss = wordnet.synsets('interest', pos='n')
for ss in iss:
    print(ss, ss.definition())
    

Synset('interest.n.01') a sense of concern with and curiosity about someone or something
Synset('sake.n.01') a reason for wanting something done
Synset('interest.n.03') the power of attracting or holding one's attention (because it is unusual or exciting etc.)
Synset('interest.n.04') a fixed charge for borrowing money; usually a percentage of the amount borrowed
Synset('interest.n.05') (law) a right or legal share of something; a financial involvement with something
Synset('interest.n.06') (usually plural) a social group whose members control some field of activity and who have common aims
Synset('pastime.n.01') a diversion that occupies one's time and thoughts (usually pleasantly)


In [24]:
# Let's create mapping from convenience
mapping = {
    'interest_1': 'interest.n.01',
    'interest_2': 'interest.n.03',
    'interest_3': 'pastime.n.01',
    'interest_4': 'sake.n.01',
    'interest_5': 'interest.n.05',
    'interest_6': 'interest.n.04',
}

#### Evaluation

- Let's use accuracy for simplicity
- Also demonstrating per-class precision, recall, and f-measure

In [25]:
from nltk.metrics.scores import precision, recall, f_measure, accuracy

refs = {k: set() for k in mapping.values()}
hyps = {k: set() for k in mapping.values()}
refs_list = []
hyps_list = []

# since WordNet defines more senses, let's restrict predictions
synsets = [ss for ss in wordnet.synsets('interest', pos='n') if ss.name() in mapping.values()]

for i, inst in enumerate(senseval.instances('interest.pos')):
    txt = [t[0] for t in inst.context]
    raw_ref = inst.senses[0] # let's get first sense
    hyp = lesk(txt, txt[inst.position], pos='n', synsets=synsets).name()
    
    ref = mapping.get(raw_ref)
    
    # for precision, recall, f-measure        
    refs[ref].add(i)
    hyps[hyp].add(i)
    
    # for accuracy
    refs_list.append(ref)
    hyps_list.append(hyp)

print("Acc:", round(accuracy(refs_list, hyps_list), 3))

for cls in hyps.keys():
    p = precision(refs[cls], hyps[cls])
    r = recall(refs[cls], hyps[cls])
    f = f_measure(refs[cls], hyps[cls], alpha=1)
    
    print("{:15s}: p={:.3f}; r={:.3f}; f={:.3f}; s={}".format(cls, p, r, f, len(refs[cls])))

Acc: 0.257
interest.n.01  : p=0.137; r=0.083; f=0.137; s=361
interest.n.03  : p=0.003; r=0.091; f=0.003; s=11
pastime.n.01   : p=0.028; r=0.242; f=0.028; s=66
sake.n.01      : p=0.059; r=0.062; f=0.059; s=178
interest.n.05  : p=0.286; r=0.104; f=0.286; s=500
interest.n.04  : p=0.547; r=0.399; f=0.547; s=1252


### Exercise
- Evaluate Original Lesk (your implementation on Senseval's `interest`)

## Supervised Learning for WSD

### Features for WSD
- Bag-of-Words (already covered)
- Collocational features

#### Bag-of-Words (BOW) Classification (recap)

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

data = [" ".join([t[0] for t in inst.context]) for inst in senseval.instances('interest.pos')]
lbls = [inst.senses[0] for inst in senseval.instances('interest.pos')]

print(data[0])
print(lbls[0])


yields on money-market mutual funds continued to slide , amid signs that portfolio managers expect further declines in interest rates .
interest_6


In [27]:
vectorizer = CountVectorizer()
classifier = MultinomialNB()
lblencoder = LabelEncoder()

stratified_split = StratifiedKFold(n_splits=5, shuffle=True)

vectors = vectorizer.fit_transform(data)

# encoding labels for multi-calss
lblencoder.fit(lbls)
labels = lblencoder.transform(lbls)

scores = cross_validate(classifier, vectors, labels, cv=stratified_split, scoring=['f1_micro'])

print(sum(scores['test_f1_micro'])/len(scores['test_f1_micro']))


0.8188401530762437


#### Collocational Features
- Assume +/-n words window from target

e.g. n=2

`... managers expect further [declines in] [interest] [rates .]`

- $w_{-1}$ : `declines`
- $w_{-2}$ : `in`
- $w_0$ __target__ : `interest`
- $w_{+1}$ : `rates`
- $w_{+2}$ : `.`

- POS-tags of these words
- word ngrams in window +/-3 are common


##### Using Collocational Features in scikit-learn
- represent features as dict
- use `DictVectorizer`

In [28]:
def collocational_features(inst):
    p = inst.position
    return {
        "w-2_word": 'NULL' if p < 2 else inst.context[p-2][0],
        "w-1_word": 'NULL' if p < 1 else inst.context[p-1][0],
        "w+1_word": 'NULL' if len(inst.context) - 1 < p+1 else inst.context[p+1][0],
        "w+2_word": 'NULL' if len(inst.context) - 1 < p+2 else inst.context[p+2][0]
    }

In [29]:
data_col = [collocational_features(inst) for inst in senseval.instances('interest.pos')]
print(data_col[0])

{'w-2_word': 'declines', 'w-1_word': 'in', 'w+1_word': 'rates', 'w+2_word': '.'}


In [30]:
from sklearn.feature_extraction import DictVectorizer
dvectorizer = DictVectorizer(sparse=False)
dvectors = dvectorizer.fit_transform(data_col)

scores = cross_validate(classifier, dvectors, labels, cv=stratified_split, scoring=['f1_micro'])

print(sum(scores['test_f1_micro'])/len(scores['test_f1_micro']))

0.8602242620494019


#### Concatenating Feature Vectors

In [31]:
import numpy as np

# let's check shape's for sanity & types (for illustration)
print(vectors.shape, type(vectors))
print(dvectors.shape, type(dvectors))

# types of CountVectorizer and DictVectorizer outputs are different 
# we need to convert them to the same format
uvectors = np.concatenate((vectors.toarray(), dvectors), axis=1)

print(uvectors.shape, type(uvectors))

(2368, 7033) <class 'scipy.sparse.csr.csr_matrix'>
(2368, 2007) <class 'numpy.ndarray'>
(2368, 9040) <class 'numpy.ndarray'>


In [32]:
# cross-validating classifier the usual way
scores = cross_validate(classifier, uvectors, labels, cv=stratified_split, scoring=['f1_micro'])

print(sum(scores['test_f1_micro'])/len(scores['test_f1_micro']))

0.8935754364367845


## Exercise
- Extend collocational features with
    - POS-tags
    - Ngrams within window
- Concatenate BOW and new collocational feature vectors & evaluate
- Evaluate Lesk and Graph-based metrics on the same split & compare